In [204]:
import random
import networkx as nx
import matplotlib.pyplot as plt

In [379]:
class Architecture:
    def __init__(self):
        self.c_capacity, self.s_capacity = 50, 10
        self.weights = [0.0125, 0.0010, 0.0200, 0.5000] # REVIEW
        self.lifetime = {0: 2, 1: 3, 2: 4, 3: 10}   # REVIEW
        self.max_actions = 10
        self.num_c = 2
        self.num_s = 6
        self.max_actions = 10

        self.data_types = []
        self.data_allocation = []
        self.data_times = []

        self.data = {}
        self.devices = {}
        self.latencies = {}
        self.max_latencies = {}

        self.last_calls = []

        self.action_device = []
        self.action_type = []

        for j in (range(len(self.weights))):
            self.data[j] = {}
            for i in range(self.num_s+self.num_c):
                self.data[j][i] = 0
                self.devices[i] = {0: 0, 1: 0, 2: 0, 3: 0}
                self.latencies[i] = {}
                self.max_latencies[i] = {}

        for i in self.latencies.keys():
            for j in self.latencies.keys():
                self.latencies[i][j] = 0
                self.max_latencies[i][j] = 0

        randomized = list(self.devices.keys())
        random.shuffle(randomized)
        bound = round(len(self.devices)/4)
        self.clusters = {
            0: randomized[:bound],
            1: randomized[bound:bound*2],
            2: randomized[bound*2:bound*3],
            3: randomized[bound*3:]
        }
        self.visualization()

    def construct_dictionaries(self):
        for i in self.data:
            for j in self.data[i]:
                self.data[i][j] = 0
        for i in self.devices:
            for j in self.devices[i]:
                self.devices[i][j] = 0
        for device, d_type in zip(self.data_allocation, self.data_types):
            self.data[d_type][device] += 1
            self.devices[device][d_type] += 1

    def update(self, data_type, device):
        self.data_types.append(data_type)
        self.data_allocation.append(device)
        self.data_times.append(0)
        self.devices[device][data_type] += 1
        self.data[data_type][device] += 1

    def heart_beat(self):
        deads = []
        for i in range(len(self.data_times)):
            self.data_times[i] += 1
            if self.data_times[i] == self.lifetime[self.data_types[i]]:
                deads.append(i)

        removed = 0
        for i in deads:
            self.devices[self.data_allocation[i-removed]][self.data_types[i-removed]] -= 1
            self.data[self.data_types[i-removed]][self.data_allocation[i-removed]] -= 1
            self.data_types.pop(i - removed)
            self.data_allocation.pop(i - removed)
            self.data_times.pop(i - removed)
            removed += 1

    def free_space(self):
        load = {}
        for i in range(self.num_c + self.num_s):
            if i < self.num_c:
                load[i] = (self.c_capacity - sum([a*b for a,b in zip(list(self.devices[i].values()), self.weights)]))/self.c_capacity
            else:
                load[i] = (self.s_capacity - sum([a*b for a,b in zip(list(self.devices[i].values()), self.weights)]))/self.s_capacity
            # if load[i] <= 0.2:
            #     self.space_flag = i
        return load

    def reduce_type(self, data_type):
        max_time = 0
        positions = []
        for i in range(len(self.data_allocation)):
            if self.data_types[i] == data_type:
                if self.data_times[i] > max_time:
                    max_time = self.data_times[i]
                    positions = []
                    positions.append(i)
                elif self.data_times[i] == max_time:
                    positions.append(i)

        removed = 0
        for i in positions:
            self.devices[self.data_allocation[i-removed]][self.data_types[i-removed]] -= 1
            self.data[self.data_types[i-removed]][self.data_allocation[i-removed]] -= 1
            self.data_times.pop(i - removed)
            self.data_types.pop(i - removed)
            self.data_allocation.pop(i -removed)
            removed += 1

    def check_quantity(self, data_type):
        if data_type != 3:
            if sum(list(self.data[data_type].values())) > 2*(self.num_s + self.num_c):
                self.reduce_type(data_type)
        else:
            if sum(list(self.data[data_type].values())) > 2*self.num_s:
                self.reduce_type(data_type)

    def visualization(self):
        keys = list(self.devices.keys())
        cs = keys[:self.num_c]
        ss = keys[self.num_c:]

        colors = []
        sizes = []

        graph = nx.Graph()

        colors.append('lightblue')
        sizes.append(1000)
        index = 0
        for i in range(len(cs)):
            if i != len(cs)-1:
                colors.append('lightblue')
                sizes.append(1000)
                graph.add_edge(cs[i],cs[i+1])
                graph.add_weighted_edges_from([(cs[i],cs[i+1],10)])
                for j in range(int(len(ss)/self.num_c)):
                    colors.append('orange')
                    sizes.append(100)
                    graph.add_edge(cs[i],ss[index])
                    graph.add_weighted_edges_from([(cs[i],ss[index],4)])
                    index += 1
            else:
                graph.add_edge(cs[i],cs[0])
                graph.add_weighted_edges_from([(cs[i],cs[0],10)])
                for j in range(int(len(ss)/self.num_c)+len(ss)%self.num_c):
                    graph.add_edge(cs[i],ss[index])
                    graph.add_weighted_edges_from([(cs[i],ss[index],4)])
                    colors.append('orange')
                    sizes.append(100)
                    index += 1

        pos = nx.spring_layout(graph)  # Position nodes using a spring layout algorithm
        # nx.draw(graph, pos, with_labels=True, node_size=sizes, node_color=colors, font_weight=12, font_color='black', edge_color='gray')
        edge_labels = nx.get_edge_attributes(graph, 'weight')
        # nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

        # plt.title("Architecture:")
        # plt.axis('off')
        # plt.show()

        for u in graph.nodes:
            shortest_paths = nx.shortest_path_length(graph, source=u, weight='weight')
            for v, weight in shortest_paths.items():
                self.latencies[u][v] = weight

        max_latency = self.latencies[self.num_c][self.num_c+self.num_s-1]
        for i in self.max_latencies.keys():
            for j in self.max_latencies[i].keys():
                self.max_latencies[i][j] = max_latency

    def compute_total_latency(self):
        total_latency = 0
        for i in range(len(self.action_type)):
            total_latency += sum([a*b for a,b in zip(self.data[self.action_type[i]].values(), self.latencies[self.action_device[i]].values())])
        return total_latency

    def compute_normalized_latency(self):
        total_latency = 0
        max_latency = 0
        for i in range(len(self.action_type)):
            total_latency += sum([a*b for a,b in zip(self.data[self.action_type[i]].values(), self.latencies[self.action_device[i]].values())])
            max_latency += sum([a*b for a,b in zip(self.data[self.action_type[i]].values(), self.max_latencies[self.action_device[i]].values())])
        return total_latency/max_latency

    def rl_update(self, new_allocation):
        self.data_allocation = new_allocation
        self.construct_dictionaries()
        return self.free_space()

    def generate(self, operation):
        if operation == 0 or operation == 1 or operation == 2:
            for i in range(self.num_s + self.num_c):
                self.update(operation,i)
            self.check_quantity(operation)
        elif operation == 3:
            for i in range(self.num_c,(self.num_c + self.num_s)):
                self.update(operation,i)
            self.check_quantity(operation)
        else:
            device=random.sample(self.clusters[operation - 10],1)
            self.action_device.append(device[0])
            self.action_type.append(operation-10)
            difference = len(self.action_device) - self.max_actions
            if difference >= 0:
                for i in range(difference):
                    self.action_device.pop(0)
                    self.action_type.pop(0)

In [306]:
# architecture = Architecture()
architecture.generate(3)
# architecture.generate(2)
architecture.heart_beat()
print(architecture.data_allocation)
print(architecture.data_types)
print(architecture.data_times)
print(architecture.latencies)
print(architecture.action_device)
print(architecture.compute_total_latency())
# print(architecture.last_calls)
# print(architecture.action_device)
# print(architecture.action_type)
# architecture.free_space()
# print(architecture.latencies)

[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 2, 3, 4, 5, 6]
[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1]
{0: {0: 0, 1: 10, 2: 4, 3: 4, 4: 4, 5: 14, 6: 14, 7: 14}, 1: {0: 10, 1: 0, 2: 14, 3: 14, 4: 14, 5: 4, 6: 4, 7: 4}, 2: {0: 4, 1: 14, 2: 0, 3: 8, 4: 8, 5: 18, 6: 18, 7: 18}, 3: {0: 4, 1: 14, 2: 8, 3: 0, 4: 8, 5: 18, 6: 18, 7: 18}, 4: {0: 4, 1: 14, 2: 8, 3: 8, 4: 0, 5: 18, 6: 18, 7: 18}, 5: {0: 14, 1: 4, 2: 18, 3: 18, 4: 18, 5: 0, 6: 8, 7: 8}, 6: {0: 14, 1: 4, 2: 18, 3: 18, 4: 18, 5: 8, 6: 0, 7: 8}, 7: {0: 14, 1: 4, 2: 18, 3: 18, 4: 18, 5: 8, 6: 8, 7: 0}}
[2]
0


In [380]:
executions = [[0,1,2,10],
              [0,1,2,11],
              [0,1,2,12],
              [0,1,2,3],
              [0,1,2,10],
              [0,1,2,11],
              [0,1,2,12],
              [0,1,2,3],
              [0,1,2,13]]
architecture = Architecture()
for i in range(3):
    for j in executions:
        for h in j:
            architecture.generate(h)
        architecture.heart_beat()
        print(architecture.compute_total_latency())
        print(1-architecture.compute_normalized_latency())
        print(architecture.free_space())
        print(architecture.devices)

64
0.5555555555555556
{0: 0.99933, 1: 0.99933, 2: 0.99665, 3: 0.99665, 4: 0.99665, 5: 0.99665, 6: 0.99665, 7: 0.99665}
{0: {0: 1, 1: 1, 2: 1, 3: 0}, 1: {0: 1, 1: 1, 2: 1, 3: 0}, 2: {0: 1, 1: 1, 2: 1, 3: 0}, 3: {0: 1, 1: 1, 2: 1, 3: 0}, 4: {0: 1, 1: 1, 2: 1, 3: 0}, 5: {0: 1, 1: 1, 2: 1, 3: 0}, 6: {0: 1, 1: 1, 2: 1, 3: 0}, 7: {0: 1, 1: 1, 2: 1, 3: 0}}
240
0.4444444444444444
{0: 0.9989100000000001, 1: 0.9989100000000001, 2: 0.9945499999999999, 3: 0.9945499999999999, 4: 0.9945499999999999, 5: 0.9945499999999999, 6: 0.9945499999999999, 7: 0.9945499999999999}
{0: {0: 1, 1: 2, 2: 2, 3: 0}, 1: {0: 1, 1: 2, 2: 2, 3: 0}, 2: {0: 1, 1: 2, 2: 2, 3: 0}, 3: {0: 1, 1: 2, 2: 2, 3: 0}, 4: {0: 1, 1: 2, 2: 2, 3: 0}, 5: {0: 1, 1: 2, 2: 2, 3: 0}, 6: {0: 1, 1: 2, 2: 2, 3: 0}, 7: {0: 1, 1: 2, 2: 2, 3: 0}}
368
0.48888888888888893
{0: 0.9989100000000001, 1: 0.9989100000000001, 2: 0.9945499999999999, 3: 0.9945499999999999, 4: 0.9945499999999999, 5: 0.9945499999999999, 6: 0.9945499999999999, 7: 0.9945499999999999